In [1]:
import qml 

In [2]:
from glob import glob
import numpy as np

In [3]:
from rdkit import Chem

In [4]:
target_xyzs = sorted(glob("../targets/*.xyz"))

In [5]:
def read_sdf(sdf):
    with open(sdf, "r") as f:
        txt = f.read().rstrip()
    return txt

In [6]:
def get_ncharges_coords(sdf):
    mol = Chem.MolFromMolBlock(sdf)
   #mol = Chem.AddHs(mol)
    # rdkit molobj
    ncharges = [atom.GetAtomicNum() for atom in mol.GetAtoms()]
    conf = mol.GetConformer()
    coords = np.asarray(conf.GetPositions())
    return ncharges, coords

In [7]:
target_files = sorted(glob("../targets/*.sdf"))
target_files

['../targets/qm9.sdf', '../targets/vitc.sdf', '../targets/vitd.sdf']

In [8]:
target_sdfs = [read_sdf(x) for x in target_files]

In [9]:
conf_data = [get_ncharges_coords(x) for x in target_sdfs]

In [10]:
ncharges_list, coords_list = zip(*conf_data)

In [11]:
# mbtypes separate to each target

In [12]:
target_reps = np.array(
[np.array(qml.representations.generate_slatm(coords_list[i], ncharges_list[i], 
                                    mbtypes=qml.representations.get_slatm_mbtypes([ncharges_list[0]]),
                                             rcut=3.5,
                                            local=True))
for i in range(len(ncharges_list))])

/home/puck/anaconda3/envs/rdkit/lib/python3.7/site-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [13]:
target_reps[2].shape

(28, 2863)

In [14]:
target_labels = [t.split("/")[-1].split(".xyz")[0] for t in target_sdfs]

In [15]:
np.savez("../representations/target_SLATM_3.5_data.npz", 
         target_labels=target_labels, 
         target_reps=target_reps, 
         target_ncharges=ncharges_list,)

/home/puck/anaconda3/envs/rdkit/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)
